# "staff action log" CSV-Datei untersuchen

Discourse erlaubt den Export von administrativen Aktionen. Für mehr Transparenz wollen wir diesen Log gerne bereitstellen. Allerdings hat sich auf den ersten Blick herausgestellt, dass durch eine vollständige Veröffentlichung die Privatsphäre einzelner Nutzer verletzt werden könnte.

In diesem Skript wird der Umfang des Logs dargestellt, sowie problematische Einträge entfernt um einen unbedenklichen Log veröffentlichen zu können.

Also denn, los geht's. Python Importfoo und erstmal die CSV-Datei laden

In [1]:
import pandas as pd
from pathlib import Path

In [2]:
DATA_DIR = Path('data')
# Export vom 12. April 2019, 20:59:55. Keine Ahnung, wofür die 7 steht
data = pd.read_csv(DATA_DIR / 'staff-action-190412-205955-7.csv')

## Was haben wir denn da

Schauen wir uns erstmal an, welche Spalten uns Discourse überhaupt liefert:

In [3]:
data.columns

Index(['staff_user', 'action', 'subject', 'created_at', 'details', 'context'], dtype='object')

Okay, WER (`staff_user`) hat WAS (`action`) mit WEM (`subject`) WANN (`created_at`) gemacht. Zusätzliche Informationen können in (`details`) stehen, und (`context`) kann beispielsweise der Link zu einem Thread sein.

Beispiel gefällig?

In [4]:
data[:1]

,staff_user,action,subject,created_at,details,context
0,soerface,entity_export,staff_action,2019-04-12 20:59:55 UTC,NaN,NaN


Der aktuellste Eintrag ist also von mir darüber, dass ich diesen Export erstellt habe. Aber wie groß ist unser Log eigentlich?

In [5]:
len(data)

1256

Und über welchen Zeitraum erstrecken sich die Logs?

In [6]:
min(data['created_at']), max(data['created_at'])

('2016-06-19 20:53:43 UTC', '2019-04-12 20:59:55 UTC')

Welche Nutzer haben überhaupt administrative Aktionen vorgenommen?

In [7]:
data['staff_user'].unique()

array(['soerface', 'cfstras', 'system', '██████', '██████',
       '██████', '██████', '██████', 'feliks', '██████', 'Wolfi',
       '██████', '██████', '██████', '██████', '██████', '██████', 'and',
       '██████', '██████', '██████', '██████', 'hemdmann', '██████',
       '0boro'], dtype=object)

Moment mal - da sind doch ein Haufen Leute dabei, die doch eigentlich gar nicht Admin sind?! Schauen wir uns mal an, was die Leute für administrative Aktionen ausgeführt haben

In [8]:
data[data['staff_user'].isin(['██████', '██████', '██████', '██████'])]

,staff_user,action,subject,created_at,details,context
151,██████,change_name,██████,2019-03-26 16:46:09 UTC,NaN,NaN
271,██████,change_name,██████,2019-01-23 10:37:04 UTC,NaN,NaN
302,██████,change_name,██████,2019-01-14 05:38:50 UTC,NaN,NaN
303,██████,change_name,██████,2019-01-14 05:38:15 UTC,NaN,NaN
311,██████,change_name,██████,2019-01-06 02:00:52 UTC,NaN,NaN
312,██████,change_name,██████,2019-01-06 02:00:50 UTC,NaN,NaN
375,██████,change_name,██████,2018-07-29 11:16:58 UTC,NaN,NaN
376,██████,change_name,██████,2018-07-29 11:14:07 UTC,NaN,NaN
406,██████,change_name,██████,2018-06-04 10:04:54 UTC,NaN,NaN
407,██████,change_name,██████,2018-06-04 10:04:37 UTC,NaN,NaN


Gut, okay - die haben mal ihren eigenen Namen geändert. Das ist schon mal keine Aktion, die wir veröffentlichen müssen, fördert nicht die Transparenz und veröffentlicht nur unnütz Nutzernamen. Ich muss nur daran denken, dieses Python-Notebook vorm pushen durch sed zu jagen, um die Namen zu schwärzen, hoffentlich vergess ich's nicht und muss dann das Repo verbrennen.

Zurück zum Thema: Wenn das keine relevanten Aktionen sind, welche haben wir denn überhaupt? Schaumermal:

In [9]:
data['action'].unique()

array(['entity_export', 'anonymize_user', 'delete_post', 'impersonate',
       'revoke_email', 'post_edit', 'change_theme', 'change_site_setting',
       'delete_topic', 'backup_destroy', 'delete_theme',
       'change_category_settings', 'change_trust_level', 'change_name',
       'post_approved', 'unsilence_user', 'silence_user',
       'create_category', 'suspend_user', 'revoke_admin', 'grant_admin',
       'activate_user', 'custom_staff', 'delete_user', 'check_email',
       'change_readonly_mode', 'backup_create', 'change_username',
       'deactivate_user', 'change_site_text', 'grant_badge',
       'grant_moderation', 'revoke_moderation', 'backup_download',
       'delete_category'], dtype=object)

Puh, das ja einiges. Gehen wir sie Stück für Stück durch und finden raus, wofür sie stehen:

Action                    | Wann sie meines Wissens nach geloggt wird |
--------------------------|-------------------------------------------|
`entity_export`           | Wenn dieses Export File erstellt wird. Unbedenklich. |
`anonymize_user`          | Wenn ein Benutzer anonymisiert wurde. Möglicherweise bedenklich, da in der `details` Spalte der original Benutzername steht, in der UI von Discourse sogar eine E-Mail Adresse, die ich im CSV file jedoch nicht finden konnte. Andererseits ist es vielleicht gerade im Sinne der Transparenz, dass bekannt wird, wenn Administratoren Konten „löschen“ (löschen geht in Discourse nicht, wenn Beiträge vorhanden sind, daher die Anonymisierungsoption) |
`delete_post`             | Wenn ein Beitrag gelöscht wird. Möglicherweise bedenklich, da auch „Gelöscht vom Verfasser“ inkludiert ist (mit Nutzername in `details`), der Nutzer ist hierbei jedoch immer „system“ und daher einfach filterbar. |
`impersonate`             | Wenn jemand sich als ein anderer Nutzer ausgibt, und damit auch potentiell Zugriff auf private Nachrichten hat. Sollte transparent gemacht werden. |
`revoke_email`            | Nicht sicher was das ist, wurde aber auch nur vom „system“ User ausgelöst. In Details steht immer „Sende keine E-Mails an 'hier_deine@emailaddres.se' bis 2018-01-23 13:37:42 UTC.“ |
`post_edit`               | Wenn ein Admin einen Beitrag editiert. Dummerweise wird auch geloggt, wenn ein Admin einen WIKI-Beitrag editiert :facepalm:. Und es ist auch nicht referenziert, welcher Beitrag es ist, lediglich der Post-Inhalt steht unter `details`. Unbedenklich, in der Form aber nutzlos und irreführend. |
`change_theme`            | Wenn am Theme rumgefriemelt wird. Unbedenklich, aber auch nicht übermäßig interessant |
`change_site_setting`     | Wenn globale Einstellungen geändert werden, wie maximale Dateigrößen ändern. Es wird nur geloggt, an welcher Option geschraubt wurde, nicht, auf was sie eingestellt war oder wurde |
`delete_topic`            | Wenn ein Thema gelöscht wird. Der Names des Thema-Authors sowie der Post-Text sind in der `detail` Spalte enthalten. |
`backup_destroy`          | Wenn ein Backup gelöscht wurde. Enthält den Dateinamen des tar-Archivs. Unbedenklich. |
`delete_theme`            | Wenn ein Theme gelöscht wurde. Unbedenklich. |
`change_category_settings`| Wenn die Einstellungen einer Kategorie geändert werden. Analog zu den globalen Einstellungen sind die eingestellten Werte nicht enthalten. Unbedenklich. |
`change_trust_level`      | Wenn das Trust-Level eines Users manuell geändert wird. Der Name des Nutzers, altes und neues Trust-Level sind sichtbar. Seh ich keinen Mehrwert drin, würde es nicht exportieren. |
`change_name`             | Wenn jemand seinen Anzeige-Namen ändert. Der alte Name steht nirgends. Enthält somit nur eine Ansammlung von Nutzernamen, würde ich nicht exportieren. |
`post_approved`           | Wenn ein Beitrag freigeschaltet wurde. Das ist bisher nur zweimal passiert, da Discourse einen unseren Nutzer fälschlicherweise als Spambot klassifizert hat und daher eine manuelle Freischaltung nötig war. Interessanterweise wurde aber nicht geloggt, welcher Beitrag denn jetzt freischgeschaltet wurde. Danke für nichts, Discourse. Unbedenklich. |
`unsilence_user`          | Siehe letzte Zeile. Die gleichen zwei Fälle. Enthält den Nutzernamen des Stummgeschalteten. |
`silence_user`            | Siehe letzte Zeile. Das System hat zweimal einen Nutzer wegen zu schnellen Tippens gesperrt. Einmal wurde ein Nutzer in der Vergangenheit gesperrt, nachdem wir uns dazu entschieden haben, das Forum exklusiv Mitgliedern vorzuenthalten (das war Mitte 2017). Enthält den Nutzernamen des Stummgeschalteten. |
`create_category`         | Wenn eine Kategorie erstellt wird. Unbedenklich. |
`suspend_user`            | Wenn ein Nutzer gesperrt wird. Das ist ein paar Mal passiert, nachdem ein Nutzer kein Mitglied mehr war. Enthält den Namen des Nutzers. |
`revoke_admin`            | Wenn jemandem Administrationsrechte entzogen werden. |
`grant_admin`             | Wenn jemandem Administrationsrechte vergeben werden. |
`activate_user`           | Wenn ein Nutzer „aktiviert“ wurde? Die Liste ist viel zu kurz, das beinhaltet definitiv nicht alle Freischaltungen. Bin mir nicht sicher, was das genau heißt. |
`custom_staff`            | Da stehen nur ein paar Git-Hashes und Dateisystempfade drin. Vielleicht, wenn jemand Updates an Discourse durchführt? Sind jedenfalls auch nur Einträge von unserem Grand-Foren-Master drin, vermutlich unbedenklich, aber imho uninteressant. |
`delete_user`             | Wenn ein Nutzer gelöscht wurde. Löschen geht nur, wenn der Nutzer keine Beiträge hat. Einige Konten wurden vom System automatisch gelöscht, da sie angelegt, aber nicht weiter genutzt wurden. |
`check_email`             | Wenn ein Administrator sich die E-Mail Adresse eines Nutzers anzeigen lässt. Enthält den Namen des Nutzers. |
`change_readonly_mode`    | Wenn das Forum in den read-only Mode geschaltet wird. Unbedenklich und uninteressant. |
`backup_create`           | Wenn ein manuelles Backup angestoßen wird. Unbedenklich. |
`change_username`         | Ähnlich wie `change_name`, nur dass es sich um den Usernamen dreht (der steht z.B. in URLs) |
`deactivate_user`         | Ok, jetzt bin ich mir unsicher, wo der Unterschied zu `suspend_user` ist. Jedenfalls auch irgend ne Form von Nutzer-Deaktivierung. Vielleicht sorgt `suspend` auch nur dafür, dass man sich nicht einloggen kann? Ach, keine Ahnung, googelt's selbst. |
`change_site_text`        | Selbsterklärend |
`grant_badge`             | BADGES! WOOHOO! Enthält den Namen des Nutzers und des Badges. |
`grant_moderation`        | Wenn Moderationsrechte vergeben werden. |
`revoke_moderation`       | Wenn Moderationsrechte entzogen werden. |
`backup_download`         | Wenn ein Backup heruntergeladen wurde. Halt nur für die UI, die regelmäßigen Backups, die cf eingerichtet hat, werden hier nicht gelistet. Unbedenklich. |
`delete_category`         | Wenn eine Kategorie gelöscht wird. Unbedenklich. |

Bevor ich jetzt Auszüge des Logs bereitstelle, lasst uns mal gemeinsam überlegen, was wir bereitstellen wollen, am besten automatisiert über die Discourse API, das sollte ja machbar sein. Wichtig ist nur, dass wir eine Whitelist an Aktionen erstellen, falls zukünftig Aktionen auftauchen, die wir noch nie ausgelöst habem.

Für einen ersten Eindruck hier aber schon mal die Zahlen, wer welche Aktion wie häufig durchgeführt hat - ohne die Change-Name Sachen, damit hier auch nur Admins gelistet werden:

In [10]:
df = data[~data['action'].isin(['change_name', 'change_username'])]
print(f'Anzahl Aktionen ohne Namensänderungen:', len(df))

with pd.option_context("display.max_rows", len(df)):
    display(df.groupby(['staff_user','action']).size().sort_values(ascending=False).to_frame())

Anzahl Aktionen ohne Namensänderungen: 1201


0
staff_user action                       
cfstras    change_site_setting       242
           check_email               129
soerface   change_theme              117
system     delete_post                87
Wolfi      check_email                59
cfstras    change_category_settings   52
soerface   change_category_settings   37
cfstras    change_theme               37
feliks     change_category_settings   36
           delete_post                35
cfstras    custom_staff               25
soerface   post_edit                  24
system     revoke_email               23
feliks     check_email                23
Wolfi      change_category_settings   23
cfstras    grant_moderation           14
           delete_post                12
Wolfi      delete_post                11
cfstras    revoke_moderation          11
           suspend_user                9
           create_category             9
Wolfi      create_category             9
           change_site_setting         9
cfstras    backup_destroy              8
           post_edit                   8
feliks     grant_badge                 8
cfstras    revoke_admin                7
           grant_admin                 7
Wolfi      backup_create               7
hemdmann   check_email                 6
cfstras    delete_user                 6
system     delete_user                 6
           change_site_setting         6
feliks     activate_user               6
cfstras    change_trust_level          5
feliks     impersonate                 5
cfstras    change_site_text            5
           activate_user               5
hemdmann   change_category_settings    4
feliks     create_category             4
           delete_topic                4
           change_site_setting         3
Wolfi      delete_category             3
cfstras    delete_topic                3
           backup_create               3
           change_readonly_mode        2
           delete_category             2
and        delete_post                 2
cfstras    deactivate_user             2
system     silence_user                2
0boro      check_email                 2
system     post_edit                   2
soerface   unsilence_user              2
           suspend_user                2
           post_approved               2
           create_category             2
           impersonate                 2
cfstras    impersonate                 2
soerface   grant_badge                 1
Wolfi      backup_download             1
system     delete_topic                1
soerface   revoke_admin                1
Wolfi      delete_topic                1
           delete_user                 1
and        change_trust_level          1
           check_email                 1
cfstras    grant_badge                 1
soerface   grant_admin                 1
           entity_export               1
           delete_topic                1
           delete_theme                1
           delete_post                 1
           deactivate_user             1
           check_email                 1
           activate_user               1
██████    grant_admin                 1
hemdmann   change_trust_level          1
feliks     delete_user                 1
           deactivate_user             1
cfstras    silence_user                1
           anonymize_user              1